# Cell counter interface

This notebook aims to provide a streamlined interface for cell counting.

In [2]:
from ipywidgets import widgets, fixed
from skimage import io, exposure, morphology
import skimage
import numpy as np
from dataclasses import dataclass
%matplotlib inline

In [12]:
@dataclass
class CounterImage():
    original_image: np.ndarray
    processed_image: np.ndarray
    channels_count: int
    channels: dict
    marks: dict

In [4]:
uploaded_file = widgets.FileUpload(multiple=False, description="Upload")
uf_name = widgets.Text(value=None,
                       placeholder='no file loaded',
                       description='',
                       disabled=True
                      )
channel_selector = widgets.Dropdown(options=['(no file loaded)'],
                                            value='(no file loaded)',
                                            description='Channel:',
                                            disabled=True
                                           )

image_store = widgets.Image()
processed_image = None
output = widgets.Output() #only used for debugging
#output_image = widgets.Image(value=b'',format='png',width='300',height='400')


def new_file_uploaded(change):
    new_name = list(change['new'])[0]
    new_img = io.imread(change['new'][new_name]['content'], plugin='imageio')
    channels = [str(i+1) for i in range(new_img.shape[-1])]
    channel_selector.options = channels
    channel_selector.disabled = False
    with output:
        print(io.imread(change['new'][new_name]['content'], plugin='imageio').shape)
        print([str(i+1) for i in range(new_img.shape[-1])])
    uf_name.value = new_name
    image_store.value = change['new'][new_name]['content']

uploaded_file.observe(new_file_uploaded, 'value')
filecontrol = widgets.HBox([uploaded_file,
                            uf_name,
                            #output,
                            channel_selector])

In [5]:
def adjust_image(image_as_bytestream,
                 lower_thresh=2, upper_thresh=98,
                 filter_size=0, 
                 channel=0,
                 cmap='viridis'
                ):
    """
    Applies contrast stretching to an image, then
    uses a white top-hat filter to remove small patches
    of brightness that would cause false positives later.
    
    Input: image; values for min and max percentile
    brightnesses to keep; size below which bright patches
    will be removed; colourmap.
    
    Output: image, hopefully with most of the background stripped
    out. If it's worked well, it'll look like bright blobs on a
    dark background.
    """
    
    try:
        channel = int(channel) - 1
        image = io.imread(image_as_bytestream, plugin='imageio')[:,:,channel]

        p2, p98 = np.percentile(image, (lower_thresh, upper_thresh))
        img_rescale = exposure.rescale_intensity(image, in_range=(p2, p98))
        selem = morphology.disk(filter_size)
        wht_tophat = morphology.white_tophat(img_rescale,selem=selem)
        io.imshow(img_rescale - wht_tophat, cmap=cmap)
        #can't believe I'm doing this
        global processed_image
        processed_image = img_rescale
    except ValueError:
        img_rescale=None
        pass
    
    return img_rescale

In [6]:
widget_style = {'description_width': 'initial'}
lower_thresh = widgets.IntSlider(min=0, max=100, step=5, value=2,
                                 description='lower threshold', style=widget_style)
upper_thresh = widgets.IntSlider(min=0, max=100, step=5, value=98,
                                 description='upper threshold', style=widget_style)
filter_size  = widgets.IntSlider(min=0, max=20, step=1, value=0,
                                 description='filter size', style=widget_style)

contrast_ui = widgets.VBox([lower_thresh, upper_thresh, filter_size])
contrast_tweaking = widgets.interactive_output(adjust_image, {'lower_thresh': lower_thresh,
                                                              'upper_thresh': upper_thresh,
                                                              'filter_size': filter_size,
                                                              'cmap': fixed('viridis'),
                                                              'image_as_bytestream': image_store,
                                                              'channel': channel_selector})

display(filecontrol, contrast_ui, contrast_tweaking)

Output()

In [8]:
type(processed_image)

numpy.ndarray

In [6]:
def detect_blobs(original_image,
                 processed_image,
                 max_sigma=30,
                 threshold=0.1
                ):
    """
    Detects bright blobs in an image using the scikit-image
    determinant of gaussian technique, then marks them on the
    image.
    
    Input: original and processed images; max_sigma to determine 
    upper limit for blob size; threshold to determine how bright
    something needs to be before it's identified as a blob.
    
    Output: displays image with red rings around detected blobs;
    returns array of blob markers (y,x,radius).
    """
    
    blobs_dog = feature.blob_dog(processed_image,
                                 max_sigma=max_sigma,
                                 threshold=threshold
                                )
    blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2) #radius calcs
    fig,axes = plt.subplots(ncols=2, figsize=(16,12))
    ax_im_pairs = list(zip(axes,
                           (processed_image,
                            original_image),
                           (True,True)
                          ))
    for ax,im,draw in ax_im_pairs:
        ax.imshow(im)
        if draw == True:
            for blob in blobs_dog:
                y,x,r = blob
                c = plt.Circle((x, y), r,
                               color='r',
                               linewidth=2,
                               fill=False
                              )
                ax.add_patch(c)
    print("{} blobs detected.".format(len(blobs_dog)))
    
    return blobs_dog